In [1]:
import re

In [13]:
# 返回一个事务列表
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [14]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [15]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [16]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [17]:
headerTable = {}
for trans in initSet: 
    for item in trans:
        headerTable[item] = headerTable.get(item, 0) + initSet[trans]

In [18]:
headerTable

{'z': 5,
 'p': 2,
 'j': 1,
 'r': 3,
 'h': 1,
 'v': 1,
 'u': 1,
 'w': 1,
 'y': 3,
 's': 3,
 'x': 4,
 't': 3,
 'n': 1,
 'o': 1,
 'q': 2,
 'e': 1,
 'm': 1}

In [19]:
for k in list(headerTable):     # 原书中代码出现错误
# https://stackoverflow.com/questions/11941817/how-to-avoid-runtimeerror-dictionary-changed-size-during-iteration-error
    if headerTable[k] < 3: 
        del(headerTable[k])

In [20]:
headerTable

{'z': 5, 'r': 3, 'y': 3, 's': 3, 'x': 4, 't': 3}

In [21]:
freqItemSet = set(headerTable.keys())
freqItemSet

{'r', 's', 't', 'x', 'y', 'z'}

In [22]:
for k in headerTable:
    headerTable[k] = [headerTable[k], None] 

In [23]:
headerTable

{'z': [5, None],
 'r': [3, None],
 'y': [3, None],
 's': [3, None],
 'x': [4, None],
 't': [3, None]}

In [24]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue       # 存放节点名字
        self.count = numOccur       # 节点出现次数
        self.nodeLink = None       # 不同项集的相同项通过nodelink连接在一起
        self.parent = parentNode   # 指向当前节点的父节点。
        # parentNode通常情况下并不需要这个变量。因为通常是从上往下迭代访问节点的。后面的内容中需要根据给定叶子节点上溯整棵树，这时就需要指向父节点的指针。
        self.children = {}         # 空字典变量，用于存放节点的子节点。
    # 两个方法：inc()对count变量增加给定值，而另一个方法disp()用于将树以文本形式显示。后者对于树构建来说并不是必要的，但是它对于调试非常有用。 
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)  # ind缩进
        for child in self.children.values():
            child.disp(ind+1)

In [26]:
retTree = treeNode('Null Set', 1, None)

In [28]:
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [29]:
freqItemSet

{'r', 's', 't', 'x', 'y', 'z'}

In [31]:
for tranSet, count in initSet.items():  # 第二次扫描数据集:根据全局频率对每个事务中的元素进行排序
    print(tranSet)
    localD = {}
    for item in tranSet:  
        if item in freqItemSet:
            localD[item] = headerTable[item][0]
            print(localD)

frozenset({'z', 'p', 'j', 'r', 'h'})
{'z': 5}
{'z': 5, 'r': 3}
frozenset({'z', 'v', 'u', 'w', 'y', 's', 'x', 't'})
{'z': 5}
{'z': 5, 'y': 3}
{'z': 5, 'y': 3, 's': 3}
{'z': 5, 'y': 3, 's': 3, 'x': 4}
{'z': 5, 'y': 3, 's': 3, 'x': 4, 't': 3}
frozenset({'z'})
{'z': 5}
frozenset({'n', 'o', 'r', 's', 'x'})
{'r': 3}
{'r': 3, 's': 3}
{'r': 3, 's': 3, 'x': 4}
frozenset({'z', 'q', 'p', 'y', 'r', 'x', 't'})
{'z': 5}
{'z': 5, 'y': 3}
{'z': 5, 'y': 3, 'r': 3}
{'z': 5, 'y': 3, 'r': 3, 'x': 4}
{'z': 5, 'y': 3, 'r': 3, 'x': 4, 't': 3}
frozenset({'z', 'q', 'y', 'e', 'm', 's', 'x', 't'})
{'z': 5}
{'z': 5, 'y': 3}
{'z': 5, 'y': 3, 's': 3}
{'z': 5, 'y': 3, 's': 3, 'x': 4}
{'z': 5, 'y': 3, 's': 3, 'x': 4, 't': 3}


In [36]:
for tranSet, count in initSet.items():  # 第二次扫描数据集:根据全局频率对每个事务中的元素进行排序
    localD = {}
    for item in tranSet:  
        if item in freqItemSet:
            localD[item] = headerTable[item][0]
    if len(localD) > 0:
        orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
        print(orderedItems)

['z', 'r']
['z', 'x', 'y', 's', 't']
['z']
['x', 'r', 's']
['z', 'x', 'y', 'r', 't']
['z', 'x', 'y', 's', 't']


In [37]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children: # orderedItems[0] 是否在 retTree.children中
        inTree.children[items[0]].inc(count) #incrament count
    else:   # 增加 items[0] 到 inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: # 更新header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:   # 对剩下的元素项迭代调用update Tree函数
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [38]:
def updateHeader(nodeToTest, targetNode):   # 不要使用递归遍历链表
    while (nodeToTest.nodeLink != None):    
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [39]:
for tranSet, count in initSet.items():  # 第二次扫描数据集:根据全局频率对每个事务中的元素进行排序
    localD = {}
    for item in tranSet:  
        if item in freqItemSet:
            localD[item] = headerTable[item][0]
    if len(localD) > 0:
        orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
        updateTree(orderedItems, retTree, headerTable, count)

In [40]:
headerTable

{'z': [5, <__main__.treeNode at 0x227383260c8>],
 'r': [3, <__main__.treeNode at 0x22737694e48>],
 'y': [3, <__main__.treeNode at 0x22736952cc8>],
 's': [3, <__main__.treeNode at 0x22738343048>],
 'x': [4, <__main__.treeNode at 0x22736952e88>],
 't': [3, <__main__.treeNode at 0x22738343148>]}

In [42]:
retTree.disp() 

   Null Set   1
     z   5
       r   1
       x   3
         y   3
           s   2
             t   2
           r   1
             t   1
     x   1
       r   1
         s   1
